In [1]:
import pandas as pd
from collections import Counter
import numpy as np
import logging
import pickle
from joblib import Parallel, delayed
import gc
log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

In [2]:
base_path = './data'
feature_path = './feature'

In [3]:
# 问题
ques = pd.read_csv(f'{base_path}/question_info_0926.txt', header=None, sep='\t')
ques.columns = ['qid', 'q_dt', 'title_t1', 'title_t2', 'desc_t1', 'desc_t2', 'topic']

# del ques['q_dt'], ques['title_t1'], ques['title_t2'], ques['desc_t1'], ques['desc_t2']

logging.info("ques %s", ques.shape)

[2019-12-06 03:29:18,827] INFO in <ipython-input-3-955e20a24f1c>: ques (1829900, 7)


In [4]:
ques[:60]

,qid,q_dt,title_t1,title_t2,desc_t1,desc_t2,topic
0,Q2234111670,D1018-H5,"SW211,SW204,SW1715,SW69,SW2033,SW138,SW57,SW13...","W22414,W963,W10458",-1,-1,"T321,T730,T5784,T4389"
1,Q760329790,D1745-H20,"SW69,SW2033,SW138,SW2616,SW2668,SW36,SW2594,SW...","W12677,W16829,W15201,W6419,W101839","SW146,SW982,SW401,SW297,SW17,SW2616,SW2668,SW3...","W1296,W2118,W12677,W16829,W15201,W6419,W101839...","T278,T12673,T4677"
2,Q741313548,D2032-H21,"SW153,SW662,SW1218,SW853,SW325,SW1056,SW467,SW...","W700,W2781,W3280,W81215","SW1956,SW3583,SW153,SW34,SW35,SW1016,SW586,SW5...","W732,W24400,W48321,W39608,W20788,W219486,W1183...",T226
3,Q3481466230,D2185-H15,"SW22,SW179,SW57,SW451,SW594,SW118,SW882,SW655,...","W3312,W1823,W1505,W638,W166,W461","SW323,SW37,SW1,SW606,SW1227,SW29,SW22,SW179,SW...","W6642,W4214,W3312,W1505,W2205,W232,W294,W7177,...","T51,T4468"
4,Q3966197028,D2269-H17,"SW1622,SW223,SW1218,SW853,SW390,SW220,SW753,SW...","W700,W895,W2253",-1,-1,"T54700,T81,T57,T17670,T43574"
5,Q3227903909,D2359-H21,"SW36,SW1010,SW1654,SW1,SW20,SW172,SW412,SW2010...","W123,W27858,W431,W505,W185,W516","SW49,SW39,SW27,SW18,SW23,SW105,SW384,SW8,SW18,...","W696,W336,W708,W1219,W505,W431,W1212,W2152,W14192","T246,T1434,T930"
6,Q3538337257,D2410-H23,"SW610,SW22,SW8,SW210,SW20,SW219,SW5,SW17,SW142...","W711,W23782,W4073,W40298,W22366",-1,-1,T39262
7,Q3882743295,D2484-H8,"SW8,SW191,SW210,SW543,SW67,SW454,SW2406,SW1,SW...","W628,W360,W2781,W3931","SW467,SW398,SW456,SW285,SW2,SW1865,SW82,SW659,...","W3280,W16201,W16947,W1522,W5026,W3020,W24942,W...","T1605,T2381,T226,T264"
8,Q1187565618,D2572-H17,"SW153,SW662,SW187,SW2909,SW2790,SW483,SW1152,S...","W459,W249383,W79,W12041,W2928,W6261,W1146,W552...","SW602,SW1167",W3622,T26028
9,Q1211094046,D2656-H11,"SW153,SW662,SW1218,SW853,SW951,SW1271,SW3992,S...","W700,W25634,W17365,W110006,W40350","SW71,SW249,SW9,SW1686,SW946,SW510,SW243,SW1309...","W109576,W9311,W3663,W200,W986,W1548,W1548,W152...","T42196,T2507,T3065,T59740,T9505"
